# 새로 만든 데이터 로드해서 시작하기 아래는 초기 코드

# 1. 라이브러리 로드

In [1]:
import pandas as pd
import numpy as np
import re
import os
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.font_manager as fm
import matplotlib as mpl
import gc
import statistics as st
plt.rcParams['font.family'] = 'Malgun Gothic'
rc={'axes.labelsize': 15, 'font.size': 15, 'legend.fontsize': 12.0, 'axes.titlesize': 20}
plt.rcParams.update(**rc)
mpl.rcParams['axes.unicode_minus'] = False
os.getcwd()

'D:\\kaggle\\ELO'

# 2. 데이터 로드 및 메모리 최적화
* train.csv: 학습 데이터
* test.csv: 테스트 데이터
* tr_train : transaction(history+new).csv에 train.csv 붙인거
* tr_test  : transaction(history+new).csv에 test.csv 붙인거

### 데이터 로드

In [2]:
%%time
train=pd.read_csv("D:/kaggle/ELO/data/new/train.csv",encoding='utf-8')
test=pd.read_csv("D:/kaggle/ELO/data/new/test.csv",encoding='utf-8')

tr_train=pd.read_csv("D:/kaggle/ELO/data/new/tr_train.csv",encoding='utf-8')
tr_test=pd.read_csv("D:/kaggle/ELO/data/new/tr_test.csv",encoding='utf-8')

Wall time: 1min 4s


### 메모리 최적화

In [3]:
def reduce_mem_usage(props, fillna=False):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                if not fillna:
                    print('[*]skip {} cause of NA value'.format(col))
                    continue
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [4]:
train, _ = reduce_mem_usage(train,fillna=False)
test, _ = reduce_mem_usage(test,fillna=False)
tr_train, _ = reduce_mem_usage(tr_train,fillna=False)
tr_test, _ = reduce_mem_usage(tr_test,fillna=False)

Memory usage of properties dataframe is : 9.243148803710938  MB
******************************
Column:  feature_1
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  feature_2
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  feature_3
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  target
dtype before:  float64
dtype after:  float32
******************************
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  4.429072380065918  MB
This is  47.91735450897139 % of the initial size
Memory usage of properties dataframe is : 5.6591339111328125  MB
******************************
Column:  feature_1
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  feature_2
dtype before:  int64
dtype after:  uint8
******************************
************

# 4. base_model 생성

#### 1) train+test(transaction) 통합 df 생성

#### 2) na 처리 / Y/N 데이터 이진화(1/0)
* category_2 / category_3 / merchant_id / first_active_month 열의 NA값 대체
* authorized_flag / categoryr_1의 Y/N을 1/0 으로 이진화

#### 3) interval_month 생성
* interval_month = purchase_date(month) - first_active_month(month)

#### 4) datetime 분리
* ex) year / month / weekday(0/1) / weekend(0/1)

#### 5) purchase_amount_log1p 생성
* np.log1p(purchase_amount) , df['purchase_amount_log1p'] 

#### 6)  id별 nunique(개수) 값 열 생성
* ex) card_id="C_ID_bd97b86450"의 city_id의 nunique가 3이면 city_id_n=3

#### 7) id별 mean or sum 인 열 생성
* ex) card_id="C_ID_bd97b86450"의 installments 평균이 1이면 installments_mean=1

#### 8) id별 min_max 인 열 생성
* ex) card_id="C_ID_bd97b86450"의 purchase_amount 최솟값이 .5444이면 installments_min=.5444

In [5]:
#train,test card_id 보존
train_id=np.unique(tr_train.card_id)
test_id=np.unique(tr_test.card_id)

### 1) train_test 통합 df 생성

In [6]:
df=pd.concat([tr_train,tr_test],axis=0).reset_index(drop=True)
df.tail()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,first_active_month,feature_1,feature_2,feature_3,target
31075387,Y,C_ID_06512aaeca,69,N,0,A,317,M_ID_f3867a48a4,1,-0.701828,2018-03-03 12:58:59,1.0,9,25,2017-10,2,2,0,NaN
31075388,Y,C_ID_8dedf7350a,174,N,1,B,222,M_ID_c0915e5912,1,-0.710844,2018-03-14 14:56:26,1.0,9,21,2017-06,2,1,0,NaN
31075389,Y,C_ID_c07707ff74,194,N,2,C,87,M_ID_7dabac675c,1,-0.476431,2018-03-29 17:41:22,5.0,21,27,2015-09,5,1,1,NaN
31075390,Y,C_ID_611d4d588f,302,N,1,B,307,M_ID_ee5a6f6a35,2,-0.731881,2018-03-11 15:07:49,3.0,7,19,2017-10,2,2,0,NaN
31075391,Y,C_ID_1320dee851,142,N,0,A,309,M_ID_7754b67f3b,2,-0.701828,2018-04-06 14:36:52,3.0,19,21,2017-08,3,1,1,NaN


### 2) na 처리 / Y/N 데이터 이진화(1/0)

In [7]:
# NA 확인 및 가장 많은 값으로 채우기
# category_2 / category_3 / merchant_id 에서 존재
# target의 NA는 test셋의 타겟이 원래 없어서 발생한 것이므로 무시 
# first_active_month 의 NA는 test셋에서 발생함... interval(구매일-첫 카드 구매일) 계산을 위해 해당 ID purchase_date의 가장 옛날값으로 대체
df.isna().sum().sort_values(ascending=False)

target                  11825698
category_2               2764609
category_3                234081
merchant_id               164697
first_active_month            55
card_id                        0
city_id                        0
category_1                     0
installments                   0
merchant_category_id           0
purchase_amount                0
month_lag                      0
feature_3                      0
purchase_date                  0
state_id                       0
subsector_id                   0
feature_1                      0
feature_2                      0
authorized_flag                0
dtype: int64

In [8]:
#category_2 / category_3 / merchant_id 에서 가장 많이 나온 값으로 NA 대체
df.category_2.fillna(df.category_2.value_counts().index[0],inplace=True)
df.category_2=df.category_2.astype('int')
df.category_3.fillna(df.category_3.value_counts().index[0],inplace=True)
df.merchant_id.fillna(df.merchant_id.value_counts().index[0],inplace=True)
df.isna().sum()

authorized_flag                0
card_id                        0
city_id                        0
category_1                     0
installments                   0
category_3                     0
merchant_category_id           0
merchant_id                    0
month_lag                      0
purchase_amount                0
purchase_date                  0
category_2                     0
state_id                       0
subsector_id                   0
first_active_month            55
feature_1                      0
feature_2                      0
feature_3                      0
target                  11825698
dtype: int64

In [9]:
# first_active_month의 NA는 id='C_ID_c27b4f80f7'에서만 발생하였으며, purchase_date의 가장 옛날값(2017-03)으로 대체
df.loc[df['first_active_month'].isna()].sort_values(by='purchase_date')

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,first_active_month,feature_1,feature_2,feature_3,target
29021171,N,C_ID_c27b4f80f7,-1,Y,10,C,511,M_ID_b9dcf28cb9,-11,-0.056289,2017-03-09 01:34:22,1,-1,7,NaN,5,2,1,NaN
29021198,N,C_ID_c27b4f80f7,-1,Y,10,C,511,M_ID_b9dcf28cb9,-11,-0.407564,2017-03-09 01:46:07,1,-1,7,NaN,5,2,1,NaN
29021174,N,C_ID_c27b4f80f7,-1,Y,10,C,511,M_ID_b9dcf28cb9,-11,-0.056289,2017-03-10 20:41:16,1,-1,7,NaN,5,2,1,NaN
29021185,N,C_ID_c27b4f80f7,-1,Y,10,C,511,M_ID_b9dcf28cb9,-11,-0.056289,2017-03-11 02:46:55,1,-1,7,NaN,5,2,1,NaN
29021219,N,C_ID_c27b4f80f7,-1,Y,10,C,511,M_ID_b9dcf28cb9,-11,-0.407564,2017-03-11 02:57:50,1,-1,7,NaN,5,2,1,NaN
29021216,Y,C_ID_c27b4f80f7,-1,Y,1,B,511,M_ID_b9dcf28cb9,-11,-0.745405,2017-03-25 00:36:49,1,-1,7,NaN,5,2,1,NaN
29021212,Y,C_ID_c27b4f80f7,-1,Y,1,B,111,M_ID_21f1585175,-9,-0.745405,2017-05-12 23:15:44,1,-1,36,NaN,5,2,1,NaN
29021177,N,C_ID_c27b4f80f7,-1,Y,6,C,111,M_ID_21f1585175,-8,-0.351109,2017-06-12 11:35:10,1,-1,36,NaN,5,2,1,NaN
29021184,N,C_ID_c27b4f80f7,-1,Y,6,C,111,M_ID_21f1585175,-8,-0.351109,2017-06-15 11:47:24,1,-1,36,NaN,5,2,1,NaN
29021201,N,C_ID_c27b4f80f7,-1,Y,1,B,111,M_ID_21f1585175,-8,-0.680941,2017-06-15 12:29:25,1,-1,36,NaN,5,2,1,NaN


In [10]:
df.first_active_month.fillna('2017-03',inplace=True)
df.loc[df.card_id=='C_ID_c27b4f80f7'].head(1)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,first_active_month,feature_1,feature_2,feature_3,target
29021167,N,C_ID_c27b4f80f7,-1,Y,1,B,111,M_ID_21f1585175,-8,-0.680941,2017-06-21 11:20:20,1,-1,36,2017-03,5,2,1,NaN


In [11]:
df.isna().sum()

authorized_flag                0
card_id                        0
city_id                        0
category_1                     0
installments                   0
category_3                     0
merchant_category_id           0
merchant_id                    0
month_lag                      0
purchase_amount                0
purchase_date                  0
category_2                     0
state_id                       0
subsector_id                   0
first_active_month             0
feature_1                      0
feature_2                      0
feature_3                      0
target                  11825698
dtype: int64

In [12]:
#authorized_flag / category_1 → N/Y 값 0/1로 교체
df.authorized_flag=df.authorized_flag.apply(lambda x : 0 if x=="N" else 1)
df.category_1=df.category_1.apply(lambda x : 0 if x=="N" else 1)

### 3) interval_month 생성

In [13]:
#interval_month 생성 (purchase_date - first_active_month)
df['purchase_date_dt']=df['purchase_date'].apply(lambda x : x[0:7])
df['purchase_date_dt']=pd.to_datetime(df['purchase_date_dt'],format='%Y-%m')
df['first_active_month_dt']=pd.to_datetime(df['first_active_month'])
df['interval_day']=df['purchase_date_dt'] - df['first_active_month_dt']

df['interval_month']=df['interval_day'].apply(lambda x: str(x)[:-14]).astype('int')//30
df.drop(columns=['purchase_date_dt','first_active_month_dt','interval_day'],inplace=True)

### 4) datetime 분리 

In [14]:
#datetime 변수 추가 (year / month )
df['purchase_date']=pd.to_datetime(df['purchase_date'])
df['year']=df['purchase_date'].dt.year
df['month']=df['purchase_date'].dt.month
df['weekday']=(df['purchase_date'].dt.dayofweek<6).astype('int')
df['weekend']=(df['purchase_date'].dt.dayofweek>=5).astype('int')

### 5) purchase_amount_log1p 생성

In [16]:
df['purchase_amount_log1p']=np.log1p(df['purchase_amount'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31075392 entries, 0 to 31075391
Data columns (total 25 columns):
authorized_flag          int64
card_id                  object
city_id                  int16
category_1               int64
installments             int16
category_3               object
merchant_category_id     int16
merchant_id              object
month_lag                int8
purchase_amount          float32
purchase_date            datetime64[ns]
category_2               int32
state_id                 int8
subsector_id             int8
first_active_month       object
feature_1                uint8
feature_2                uint8
feature_3                uint8
target                   float64
interval_month           int32
year                     int64
month                    int64
weekday                  int32
weekend                  int32
purchase_amount_log1p    float32
dtypes: datetime64[ns](1), float32(2), float64(1), int16(3), int32(4), int64(4), int8(3), obje

In [17]:
df.tail(3)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,feature_1,feature_2,feature_3,target,interval_month,year,month,weekday,weekend,purchase_amount_log1p
31075389,1,C_ID_c07707ff74,194,0,2,C,87,M_ID_7dabac675c,1,-0.476431,...,5,1,1,NaN,30,2018,3,1,0,-0.647086
31075390,1,C_ID_611d4d588f,302,0,1,B,307,M_ID_ee5a6f6a35,2,-0.731881,...,2,2,0,NaN,5,2018,3,0,1,-1.316325
31075391,1,C_ID_1320dee851,142,0,0,A,309,M_ID_7754b67f3b,2,-0.701828,...,3,1,1,NaN,8,2018,4,1,0,-1.210086


In [21]:
df.to_csv("D:/kaggle/ELO/data/new/df.csv",encoding='utf-8',index=None)

# 기존 변수별 생성할 파생변수 정리

### 01. authorized_flag 
* authroized_flag_sum  : ID별 승인된 거래 개수 
* authroized_flag_mean : ID별 평균 승인률

### 02. card_id
* card_id_count : ID별 나타난 transaction의 개수 

### 03. city_id
* city_id_nunique : ID별 city_id 개수

### 04. category_1
* category_1_sum: ID별 Y의 개수 
* category_1_mean: ID별 평균 Y 비율 

### 05. installments
* installments_min  : ID별 최소 할부개월
* installments_max  : ID별 최대 할부개월
* installments_mean : ID별 평균 할부개월


### 06. category_3
* category_3_nunique : ID별 category_3 개수

### 07. merchant_category_id
* merchant_category_id_nunique : ID별 merchant_category_id 개수

### 08. merchant_id
* merchant_id_nunique : ID별 merchant_id 개수

### 09. month_lag
* month_lag_min  : ID별 기준일 시차 최소(ID별로 기준일이 다른듯 하다)
* month_lag_max  : ID별 기준일 시차 최대(ID별로 기준일이 다른듯 하다)
* month_lag_mean : ID별 기준일 시차 평균(ID별로 기준일이 다른듯 하다)

### 10. purchase_amount(log1p 로 진행)
* purchase_amount_log1p_min : ID별 최소 사용 금액 
* purchase_amount_log1p_max : ID별 최대 사용금액
* purchase_amount_log1p_sum : ID별 사용 금액 합
* purchase_amount_log1p_mean: ID별 평균 사용금액

### 11.  purchase_date
* purchase_date_min: ID별 가장 과거 구매일
* purchase_date_max: ID별 최근 구매일 

* year_nunique: ID별 구매 년도 개수
* month_nunique: ID별 구매 월 개수
* weekday_sum: ID별 총 평일 구매 횟수 
* weekday_mean: ID별 평일 구매 비율
* weekend_sum: ID별 총 주말 구매 횟수 
* weekend_mean: ID별 주말 구매 비율

### 12. category_2 
* category_2_nunique: ID별 category_2 개수

### 13. state_id
* state_id_nunique: ID별 state_id 개수

### 14. subsector_id
* subsector_id_nunique: ID별 subsector_id 개수

### 15. interval_month
* interval_month_min : ID별 구매일과 첫 구매일 시차의 최소
* interval_month_max : ID별 구매일과 첫 구매일 시차의 최대 
* interval_month_mean: ID별 구매일과 첫 구매일 시차의 평균 

In [24]:
%%time
train=pd.read_csv("D:/kaggle/ELO/data/new/train.csv",encoding='utf-8')
test=pd.read_csv("D:/kaggle/ELO/data/new/test.csv",encoding='utf-8')
df=pd.read_csv("D:/kaggle/ELO/data/new/df.csv",encoding='utf-8')

train, _ = reduce_mem_usage(train,fillna=False)
test, _ = reduce_mem_usage(test,fillna=False)
df, _ = reduce_mem_usage(df,fillna=False)

df['purchase_date']=pd.to_datetime(df['purchase_date'])

Memory usage of properties dataframe is : 9.243148803710938  MB
******************************
Column:  feature_1
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  feature_2
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  feature_3
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  target
dtype before:  float64
dtype after:  float32
******************************
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  4.429072380065918  MB
This is  47.91735450897139 % of the initial size
Memory usage of properties dataframe is : 5.6591339111328125  MB
******************************
Column:  feature_1
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  feature_2
dtype before:  int64
dtype after:  uint8
******************************
************

In [26]:
def aggregate_func(dataframe):
    
    agg_func = {

    'authorized_flag': ['sum', 'mean'],
    'city_id': ['nunique'],
    'category_1': ['sum','mean'],
    'installments': ['min','max','mean'],
    'category_3': ['nunique'],
    'merchant_category_id': ['nunique'],
    'merchant_id': ['nunique'],
    'month_lag': ['min','max','mean'],
    'purchase_amount_log1p': ['min','max','sum','mean'],
    'purchase_date': ['min','max'],
    'year': ['nunique'],
    'month': ['nunique'],
    'weekday': ['sum','mean'],
    'weekend': ['sum','mean'],
    'category_2': ['nunique'],
    'state_id': ['nunique'],
    'subsector_id': ['nunique'],
    'interval_month': ['min','max','mean']                 
    }
    
    agg_dataframe = dataframe.groupby(['card_id']).agg(agg_func)
    agg_dataframe.columns = ['_'.join(col).strip() for col in agg_dataframe.columns.values]
    agg_dataframe.reset_index(inplace=True)
    
    ref_df = (dataframe.groupby('card_id')
          .size()
          .reset_index(name='card_id_count'))
    
    agg_dataframe = pd.merge(ref_df, agg_dataframe, on='card_id', how='left')
    
    return agg_dataframe

In [27]:
df_new = aggregate_func(df)
df_new.tail(3)

,card_id,card_id_count,authorized_flag_sum,authorized_flag_mean,city_id_nunique,category_1_sum,category_1_mean,installments_min,installments_max,installments_mean,...,weekday_sum,weekday_mean,weekend_sum,weekend_mean,category_2_nunique,state_id_nunique,subsector_id_nunique,interval_month_min,interval_month_max,interval_month_mean
325537,C_ID_ffff756266,25,15.0,0.600000,2,3.0,0.120000,-1,12,3.160000,...,23.0,0.920000,2.0,0.080000,1,2,10,3,18,7.840000
325538,C_ID_ffff828181,198,174.0,0.878788,12,11.0,0.055556,-1,4,1.202020,...,185.0,0.934343,47.0,0.237374,4,7,24,24,39,31.232323
325539,C_ID_fffffd5772,87,78.0,0.896552,4,33.0,0.379310,-1,3,1.034483,...,78.0,0.896552,21.0,0.241379,2,3,10,0,7,4.160920


In [32]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325540 entries, 0 to 325539
Data columns (total 34 columns):
card_id                         325540 non-null object
card_id_count                   325540 non-null int64
authorized_flag_sum             325540 non-null float64
authorized_flag_mean            325540 non-null float64
city_id_nunique                 325540 non-null int64
category_1_sum                  325540 non-null float64
category_1_mean                 325540 non-null float64
installments_min                325540 non-null int16
installments_max                325540 non-null int16
installments_mean               325540 non-null float64
category_3_nunique              325540 non-null int64
merchant_category_id_nunique    325540 non-null int64
merchant_id_nunique             325540 non-null int64
month_lag_min                   325540 non-null int8
month_lag_max                   325540 non-null int8
month_lag_mean                  325540 non-null float64
purchase_amoun

In [31]:
train_test=pd.concat([train,test],axis=0).reset_index(drop=True)
train_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325540 entries, 0 to 325539
Data columns (total 6 columns):
first_active_month    325539 non-null object
card_id               325540 non-null object
feature_1             325540 non-null uint8
feature_2             325540 non-null uint8
feature_3             325540 non-null uint8
target                201917 non-null float32
dtypes: float32(1), object(2), uint8(3)
memory usage: 7.1+ MB


In [33]:
df_base=pd.merge(df_new,train_test,on='card_id',how='left')
df_base.tail(3)

,card_id,card_id_count,authorized_flag_sum,authorized_flag_mean,city_id_nunique,category_1_sum,category_1_mean,installments_min,installments_max,installments_mean,...,state_id_nunique,subsector_id_nunique,interval_month_min,interval_month_max,interval_month_mean,first_active_month,feature_1,feature_2,feature_3,target
325537,C_ID_ffff756266,25,15.0,0.600000,2,3.0,0.120000,-1,12,3.160000,...,2,10,3,18,7.840000,2016-10,5,1,1,NaN
325538,C_ID_ffff828181,198,174.0,0.878788,12,11.0,0.055556,-1,4,1.202020,...,7,24,24,39,31.232323,2015-01,2,1,0,-0.600063
325539,C_ID_fffffd5772,87,78.0,0.896552,4,33.0,0.379310,-1,3,1.034483,...,3,10,0,7,4.160920,2017-08,2,1,0,-1.073077


In [34]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325540 entries, 0 to 325539
Data columns (total 39 columns):
card_id                         325540 non-null object
card_id_count                   325540 non-null int64
authorized_flag_sum             325540 non-null float64
authorized_flag_mean            325540 non-null float64
city_id_nunique                 325540 non-null int64
category_1_sum                  325540 non-null float64
category_1_mean                 325540 non-null float64
installments_min                325540 non-null int16
installments_max                325540 non-null int16
installments_mean               325540 non-null float64
category_3_nunique              325540 non-null int64
merchant_category_id_nunique    325540 non-null int64
merchant_id_nunique             325540 non-null int64
month_lag_min                   325540 non-null int8
month_lag_max                   325540 non-null int8
month_lag_mean                  325540 non-null float64
purchase_amoun

In [35]:
#train,test card_id 보존
train_id=np.unique(tr_train.card_id)
test_id=np.unique(tr_test.card_id)

In [36]:
df_train=df_base.loc[df_base.card_id.isin(train_id)]
df_test=df_base.loc[df_base.card_id.isin(test_id)]
df_test=df_test.drop(columns=['target'])

In [37]:
df_train.tail(1)

,card_id,card_id_count,authorized_flag_sum,authorized_flag_mean,city_id_nunique,category_1_sum,category_1_mean,installments_min,installments_max,installments_mean,...,state_id_nunique,subsector_id_nunique,interval_month_min,interval_month_max,interval_month_mean,first_active_month,feature_1,feature_2,feature_3,target
325539,C_ID_fffffd5772,87,78.0,0.896552,4,33.0,0.37931,-1,3,1.034483,...,3,10,0,7,4.16092,2017-08,2,1,0,-1.073077


In [38]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201917 entries, 0 to 325539
Data columns (total 39 columns):
card_id                         201917 non-null object
card_id_count                   201917 non-null int64
authorized_flag_sum             201917 non-null float64
authorized_flag_mean            201917 non-null float64
city_id_nunique                 201917 non-null int64
category_1_sum                  201917 non-null float64
category_1_mean                 201917 non-null float64
installments_min                201917 non-null int16
installments_max                201917 non-null int16
installments_mean               201917 non-null float64
category_3_nunique              201917 non-null int64
merchant_category_id_nunique    201917 non-null int64
merchant_id_nunique             201917 non-null int64
month_lag_min                   201917 non-null int8
month_lag_max                   201917 non-null int8
month_lag_mean                  201917 non-null float64
purchase_amoun

In [39]:
df_test.tail(1)

,card_id,card_id_count,authorized_flag_sum,authorized_flag_mean,city_id_nunique,category_1_sum,category_1_mean,installments_min,installments_max,installments_mean,...,category_2_nunique,state_id_nunique,subsector_id_nunique,interval_month_min,interval_month_max,interval_month_mean,first_active_month,feature_1,feature_2,feature_3
325537,C_ID_ffff756266,25,15.0,0.6,2,3.0,0.12,-1,12,3.16,...,1,2,10,3,18,7.84,2016-10,5,1,1


In [40]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123623 entries, 1 to 325537
Data columns (total 38 columns):
card_id                         123623 non-null object
card_id_count                   123623 non-null int64
authorized_flag_sum             123623 non-null float64
authorized_flag_mean            123623 non-null float64
city_id_nunique                 123623 non-null int64
category_1_sum                  123623 non-null float64
category_1_mean                 123623 non-null float64
installments_min                123623 non-null int16
installments_max                123623 non-null int16
installments_mean               123623 non-null float64
category_3_nunique              123623 non-null int64
merchant_category_id_nunique    123623 non-null int64
merchant_id_nunique             123623 non-null int64
month_lag_min                   123623 non-null int8
month_lag_max                   123623 non-null int8
month_lag_mean                  123623 non-null float64
purchase_amoun

In [ ]:
################################################### 끝 여기 밑에는 테스트 코드 ######################################################### 

## nunique 열
* ['city_id','category_3','merchant_category_id','merchant_id','year','month','category_2','state_id','subsector_id']


In [27]:
#nunique 열 리스트 설정
n_unique_col=['city_id','category_3','merchant_category_id','merchant_id','year','month','category_2','state_id','subsector_id']

#nunique 열 생성
for i in range(len(n_unique_col)):
    nunique_basket=[]
   
    for j in range(len(df_new.card_id)):
        value=df.loc[df.card_id==df_new.card_id[j],n_unique_col[i]].nunique()
        nunique_basket.append(value)
        
    df_new['{}'.format(str(n_unique_col[i])+"_"+"nunique")]=nunique_basket

KeyboardInterrupt: 

## sum 열
* ['authorized_flag','category_1','weekday','weekend']

In [28]:
temp_df = df.groupby(['card_id']).agg(agg_func)

StatisticsError: no unique mode; found 2 equally common values

## mean 열
* ['authorized_flag','category_1','installments','month_lag','purchase_amount','weekday','weekend','interval_month']

## min 열

## max 열

## count 열

In [22]:
temp_df=df.loc[:10000]
temp_df.tail()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,...,first_active_month,feature_1,feature_2,feature_3,target,interval_month,year,month,weekday,weekend
9996,1,C_ID_cf58d5be10,17,0,1,B,818,M_ID_4bcf17b639,-7,-0.506484,...,2016-10,5,1,1,0.854866,9,2017,7,0,1
9997,1,C_ID_cf58d5be10,73,0,1,B,818,M_ID_852a674b3e,-12,-0.683045,...,2016-10,5,1,1,0.854866,4,2017,2,1,0
9998,1,C_ID_cf58d5be10,331,0,1,B,307,M_ID_b2fb4f93d1,-4,-0.637936,...,2016-10,5,1,1,0.854866,12,2017,10,1,0
9999,1,C_ID_cf58d5be10,166,0,1,B,307,M_ID_4bf61484df,-8,-0.518445,...,2016-10,5,1,1,0.854866,8,2017,6,1,0
10000,1,C_ID_cf58d5be10,200,0,1,B,278,M_ID_2f65280a18,-11,-0.682594,...,2016-10,5,1,1,0.854866,5,2017,3,1,0


In [23]:
new_df=pd.DataFrame(temp_df.card_id.unique(),columns=['card_id'])
new_df.head()

,card_id
0,C_ID_5037ff576e
1,C_ID_0e171c1b48
2,C_ID_fc8e41b9cf
3,C_ID_b271e7ab60
4,C_ID_4bed29d75c


In [24]:
n_unique_col=['city_id','category_3','merchant_category_id','merchant_id','year','month','category_2','state_id','subsector_id']
for i in range(len(n_unique_col)):
    temp=[]
   
    for j in range(len(new_df.card_id)):
        value=temp_df.loc[temp_df.card_id==new_df.card_id[j],n_unique_col[i]].nunique()
        temp.append(value)
        
    new_df['{}'.format(str(n_unique_col[i])+"_"+"nunique")]=temp

In [25]:
new_df.tail()

,card_id,city_id_nunique,category_3_nunique,merchant_category_id_nunique,merchant_id_nunique,year_nunique,month_nunique,category_2_nunique,state_id_nunique,subsector_id_nunique
35,C_ID_1964c39b55,9,2,42,95,2,9,5,7,25
36,C_ID_65188857e6,9,3,42,101,2,8,1,3,19
37,C_ID_b86ad6f26b,8,2,30,78,2,6,2,3,16
38,C_ID_542f1fa442,2,2,24,49,2,12,1,2,17
39,C_ID_cf58d5be10,14,1,6,21,1,8,5,9,5


In [15]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 8 columns):
card_id                         40 non-null object
city_id_nunique                 40 non-null int64
category_3_nunique              40 non-null int64
merchant_category_id_nunique    40 non-null int64
merchant_id_nunique             40 non-null int64
category_2_nunique              40 non-null int64
state_id_nunique                40 non-null int64
subsector_id_nunique            40 non-null int64
dtypes: int64(7), object(1)
memory usage: 2.6+ KB
